# SMIB Control

In [49]:
%matplotlib widget

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sopt
import ipywidgets
from pydae import ssa
import scipy.signal as sctrl
import control as ctrl

## Import system module

In [51]:
from smib_4ord_ctrl import smib_4ord_ctrl_class

## Instantiate system

In [52]:
syst = smib_4ord_ctrl_class()
syst.t_end = 30.0
syst.Dt = 0.001
syst.decimation =10
syst.update()

## Initialize the system (backward and foreward)

In [53]:
events=[{'p_t':1, 'v_1':1}]
syst.initialize(events,xy0=1)
print(syst.get_value('p_m_ref'))
print(syst.get_value('v_f'))
p_t = 0.5
v_1 = 0.9
events=[{'p_t':p_t, 'v_1':v_1}]
syst.initialize(events,xy0='prev')
print(syst.get_value('p_m_ref'))
print(syst.get_value('v_f'))

1.00300187669663
2.0914374147123884
0.5128334896347432
2.887333199806721


\begin{equation}
\mathbf f = \left[\begin{matrix}\Omega_{b} \left(\omega - \omega_{s}\right)\\\frac{- D \left(\omega - \omega_{s}\right) - p_{e} + p_{m}}{2 H}\\\frac{- e'_q - i_{d} \left(- X'_d + X_{d}\right) + v_{f}}{T'_{d0}}\\\frac{- e'_d + i_{q} \left(- X'_q + X_{q}\right)}{T'_{q0}}\\\frac{- p_{m} + p_{m}^\star}{T_{g}}\end{matrix}\right]
\;\;\;\;
\mathbf x = \left[\begin{matrix}\delta\\\omega\\e'_q\\e'_d\\p_{m}\end{matrix}\right]
\end{equation}
\begin{equation}
\mathbf g = \left[\begin{matrix}v_{1} \sin{\left(\delta - \theta_{1} \right)} - v_{d}\\v_{1} \cos{\left(\delta - \theta_{1} \right)} - v_{q}\\i_{d} \left(R_{a} i_{d} + v_{d}\right) + i_{q} \left(R_{a} i_{q} + v_{q}\right) - p_{e}\\q_{t} + \frac{v_{0} v_{1} \cos{\left(\theta_{0} - \theta_{1} \right)}}{X_{line}} - \frac{v_{1}^{2}}{X_{line}}\\R_{a} i_{q} + X'_d i_{d} - e'_q + v_{q}\\R_{a} i_{d} - X'_q i_{q} - e'_d + v_{d}\\p_{t} + \frac{v_{0} v_{1} \sin{\left(\theta_{0} - \theta_{1} \right)}}{X_{line}}\\q_{t} + \frac{v_{0} v_{1} \cos{\left(\theta_{0} - \theta_{1} \right)}}{X_{line}} - \frac{v_{1}^{2}}{X_{line}}\\i_{d} v_{d} + i_{q} v_{q} - p_{t}\\i_{d} v_{q} - i_{q} v_{d} - q_{t}\end{matrix}\right]
\;\;\;\;
\mathbf y^{ini} = \left[\begin{matrix}v_{d}\\v_{q}\\p_{e}\\i_{d}\\i_{q}\\q_{t}\\\theta_{1}\\p_{m}^\star\\v_{f}\end{matrix}\right]
\;\;\;\;
\mathbf y^{run} = \left[\begin{matrix}v_{d}\\v_{q}\\p_{e}\\i_{d}\\i_{q}\\q_{t}\\\theta_{1}\\p_{t}\\v_{1}\end{matrix}\right]
\end{equation}

\begin{equation}
\mathbf A_e = 
\left[\begin{matrix}
\mathbf A_p & \mathbf 0 \\
 - \mathbf C_p & -\mathbf I \\
\end{matrix}
\right]
\;\;\;\;
\mathbf B_e =
\left[\begin{matrix}
\mathbf B_p   \\
 \mathbf 0  \\
\end{matrix}
\right]
\end{equation}

in discrete $\mathbf A_d$, $\mathbf B_d$  :

\begin{equation}
\mathbf A_d = 
\left[\begin{matrix}
\mathbf A_p & \mathbf 0 \\
 - \mathbf C_p & -\mathbf I \\
\end{matrix}
\right]
\;\;\;\;
\mathbf B_d =
\left[\begin{matrix}
\mathbf B_d   \\
 \mathbf 0  \\
\end{matrix}
\right]
\end{equation}

In [59]:
Ts_control = 0.1
Δt = Ts_control
x_d_ctrl_list = ['delta','omega','e1q','e1d','p_m']   # states to consider in the reduction
z_ctrl_list = ['p_t','v_1']                           # outputs to consider in the controller
z_obs_list =  ['omega','p_m','p_t','q_t','v_1','theta_1','i_t','e1d']       # outputs to consider in the observer

syst.Δt = Δt

## Calculate equilibirum point
p_t = 0.5
v_1 = 1.0
events=[{'p_t':p_t, 'v_1':v_1}]
syst.initialize(events,xy0=1)
print(syst.get_value('p_m_ref'))
print(syst.get_value('v_f'))
ssa.eval_ss(syst)

z_ctrl_idxs = [syst.outputs_list.index(item) for item in z_ctrl_list]
z_obs_idxs  = [syst.outputs_list.index(item) for item in z_obs_list]

# linear continous plant
A_p = syst.A
B_p = syst.B
C_p = syst.C
D_p = syst.D

N_z_p,N_x_p = C_p.shape
N_x_p,N_u_p = B_p.shape

x_ctrl_keep_idx = [syst.x_list.index(item)  for item in x_d_ctrl_list]
x_ctrl_elim_idx = list(set(range(N_x_p)) - set(x_ctrl_keep_idx)) 

# Reduction and discretization ##################################################################
if len(x_ctrl_elim_idx) == 0:
    # without reduction:
    A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_p,B_p,C_p,D_p),Ts_control,method='zoh')
    T_r = np.eye(N_x_p)

else:
    # with reduction:
    sys = ctrl.ss(A_p, B_p, C_p, D_p)
    T_r =np.eye(N_x_p)
    
    rsys = ctrl.modred(sys, x_ctrl_elim_idx, method='truncate')
    A_pr,B_pr,C_pr,D_pr = rsys.A,rsys.B,rsys.C,rsys.D
    A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_pr,B_pr,C_pr,D_pr),Ts_control,method='zoh')
    T_r = np.delete(np.eye(N_x_p),x_ctrl_elim_idx,axis=0)
    
N_z_d,N_x_d = C_d.shape
N_x_d,N_u_d = B_d.shape

syst.T_r = T_r  # Full linear discrete states to reduced (truncated) states 

syst.A_d = A_d
syst.B_d = B_d


# Controller ##################################################################################
C_c = C_d[z_ctrl_idxs,:]
D_c = D_d[z_ctrl_idxs,:]

N_z_c,N_x_c = C_c.shape

O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)

# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# delay in the input:
# Δx_d = A_d*Δx_d + B_d*Δx_r
# Δz_c = C_c*Δx_c 
# Δx_r = Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δx_r
# Δx_r = Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref

A_e = np.block([
                [    A_d,  B_d, O_xu],    # Δx_d
                [   O_ux, O_uu, O_uu],    # Δx_r
                [ Δt*C_c, O_uu, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [   I_uu],
                [   O_uu],    
               ])

# weighting matrices
Q_c = np.eye(A_e.shape[0])
idx = x_d_ctrl_list.index('delta')
Q_c[idx,idx] = 1
idx = x_d_ctrl_list.index('omega')
Q_c[idx,idx] = 1e1
Q_c[-2,-2] = 1e1
Q_c[-1,-1] = 1e4

R_c = np.diag([1,1]) 

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)
E_c = np.log(E_c)/Δt

syst.x_ctrl_keep_idx = x_ctrl_keep_idx
syst.x_ctrl_elim_idx = x_ctrl_elim_idx
syst.z_ctrl_list = z_ctrl_list

syst.N_z_c = N_z_c
syst.N_u_d = N_u_d

syst.x_ctrl = np.zeros((N_u_d+N_z_d,1))
syst.K_c = K_c
syst.C_c = C_c
syst.D_c = D_c

# Observer ###########################################################

# discretized plant:
# Dx_d = A_d*Dx_d + B_d*Du_d
# z_o  = C_o*Dx_d + D_o*Du_d

# x_o = A_d*x_o + B_d*u_d + L_o*(z_o - C_o*x_o - D_o*Du_d)

C_o = C_d[z_obs_idxs,:]
D_o = D_d[z_obs_idxs,:]

N_z_o = C_o.shape[0]

Q_o = np.eye(A_d.shape[0])*100000
idx = x_d_ctrl_list.index('e1q')
Q_c[idx,idx] = 1e10

R_o = np.diag([1]*N_z_o)
K_o_T,S_o,E_o = ssa.dlqr(A_d.T,C_o.T,Q_o,R_o)
K_o = K_o_T.T

syst.K_o = K_o
syst.C_o = C_o
syst.D_o = D_o

syst.z_obs_list = z_obs_list
syst.N_z_o = N_z_o
syst.x_obs = np.zeros((N_x_d,1))

print('damp_ctrl',-E_c.real/np.abs(E_c))
print('damp_obs',-E_o.real/np.abs(E_o))

0.500750117216703
1.3580999148420672
damp_ctrl [0.06972038 0.06972038 1.         0.75091716 0.75091716 0.79722787
 0.79722787 1.         0.99643631]
damp_obs [-1. -1. -1. -1. -1.]


## Controller and observer

In [60]:
def control(syst,z_ref):
    
    Δt = syst.Δt                    # disctretization time
    T_r = syst.T_r                  # Full linear discrete states to reduced (truncated) states
    
    x_d = T_r @ syst.struct[0].x    # plant dynamic states (discrete)
    x_d_0 = syst.x_d_0              # plant initial steady state (discrete)
    Δx_d = x_d - x_d_0              # plant dynamic states increment (discrete)
    
    x_r_0  = syst.x_r_0              # initial delays values
    
    Δx_e = np.copy(syst.Δx_e)     # extended plant dynamic states increment (discrete)

    u_d = np.copy(syst.u_d)       # plant inputs
    u_d_0 = syst.u_d_0              # plant initial steady inputs (discrete)
    Δu_d = syst.Δu_d              # plant inputs increment (discrete)

    z_c_0 = np.copy(syst.z_c_0)     # controller considered initial plant ouputs
    z_o_0 = np.copy(syst.z_o_0)     # observer considered initial plant ouputs
    
    A_d = syst.A_d            # plant A matrix without delay and integrators extension (discrete)
    B_d = syst.B_d            # plant B matrix without delay and integrators extension (discrete)
    C_c = syst.C_c            # controller considered plant C matrix (discrete)
    D_c = syst.D_c            # controller considered plant D matrix (discrete) 
    C_o = syst.C_o            # observer considered plant C matrix (discrete) 
    D_o = syst.D_o            # observer considered plant D matrix (discrete)   
    
    N_x_d,N_u_d = B_d.shape  # dimensions
    N_z_c,N_x_c = C_c.shape
    N_z_o,N_x_o = C_o.shape    
    
    K_c = syst.K_c            # controller gains matrix
    K_o = syst.K_o            # observer gains matrix
    
    Δx_c = np.copy(syst.x_ctrl)  # controler states
    Δx_r = Δx_c[:syst.N_u_d,:]   # delay discrete states
    Δx_i = Δx_c[syst.N_u_d:,:]   # integrators discrete states
    
    Δx_o = syst.x_obs       # observer states

     
    # outputs to control         
    z_ctrl_values_list = []
    for item in syst.z_ctrl_list:
         z_ctrl_values_list += [syst.get_value(item)]
    z_c = np.array(z_ctrl_values_list).reshape(N_z_c,1)
    
    Δz_c =   (z_c-z_c_0) - (z_ref-z_c_0)
    
    
    # outputs for observer         
    z_obs_values_list = []
    for item in syst.z_obs_list:
         z_obs_values_list += [syst.get_value(item)]
    z_o = np.array(z_obs_values_list).reshape(N_z_o,1)
    
    Δz_o = z_o - z_o_0
    

    # control dynamics
    Δx_r_k1 = Δu_d
    Δx_i_k1 = Δx_i + Δt*Δz_c   

    
    # control law  
    Δu_d = -K_c @ Δx_e   

    # observer dynamics
    Δx_o_m1 = A_d @ Δx_o + B_d@Δu_d  +  K_o @ (Δz_o - C_o @ Δx_o - D_o @ Δu_d)
    
    # save statates for next step
    syst.x_ctrl[:syst.N_u,:] = np.copy(Δx_r_k1)
    syst.x_ctrl[syst.N_u:,:] = np.copy(Δx_i_k1)
    syst.x_obs = np.copy(Δx_o_m1)
    
    if syst.observer == False:
        syst.Δx_e  = np.copy(np.block([[Δx_d],[Δx_r_k1],[Δx_i_k1]]))
        
    if syst.observer == True:    
        syst.Δx_e  = np.copy(np.block([[Δx_o_m1],[Δx_r_k1],[Δx_i_k1]]))
        
    syst.Δu_d = Δu_d
    
    # outputs for post-processing
    syst.z_obs = C_o @ Δx_o + D_o @ Δu_d
       
    return Δx_r + x_r_0

## Initialization

In [61]:
times = np.arange(0.0,20,Δt)

## Calculate initial references
p_t = 0.5
v_1 = 1.0
events=[{'p_t':p_t, 'v_1':v_1}]
syst.initialize(events,xy0=1.0)
syst.xy_prev[0]
syst.initialize(events,xy0=1.0)

# initial inputs
u_values_list = []
for item in syst.u_run_list:
     u_values_list += [syst.get_value(item)]
u_d_0 = np.array(u_values_list).reshape(syst.N_u_d,1)

# initial states
x_d_0 = T_r @ syst.struct[0].x    # initial plant states
x_r_0 = u_d_0                     # initial delay states
x_i_0 = np.zeros((syst.N_u_d,1))  # initial integrator states


Δx_e = np.block([[x_d_0*0],[x_r_0*0],[x_i_0*0]])

# initial outputs
z_ctrl_values_list = []
for item in syst.z_ctrl_list:
     z_ctrl_values_list += [syst.get_value(item)]
z_c_0 = np.array(z_ctrl_values_list).reshape(N_z_c,1)

# outputs for observer         
z_obs_values_list = []
for item in syst.z_obs_list:
     z_obs_values_list += [syst.get_value(item)]
z_o_0 = np.array(z_obs_values_list).reshape(N_z_o,1)

syst.Δx_e = Δx_e
syst.Δu_d = np.zeros((syst.N_u_d,1))
syst.x_d_0 = x_d_0
syst.x_r_0 = x_r_0
syst.u_d_0 = u_d_0
syst.z_c_0 = z_c_0
syst.z_o_0 = z_o_0

syst.u_d =  np.copy(u_d_0)
syst.x_ctrl = np.block([[x_r_0],[x_i_0]])*0
syst.x_obs = np.copy(x_d_0)*0

syst.observer = False

## Simulation 

In [62]:
T_ctrl = np.zeros((len(times),1))
X_ctrl = np.zeros((len(times),N_u_d*2))
U_ctrl = np.zeros((len(times),2))
X_obs = np.zeros((len(times),N_x_d))
Z_obs = np.zeros((len(times),N_z_o))
it = 0
for t in times:
    
    # references
    p_t_ref = 0.5
    v_1_ref = 1.0
   
    if t>1:
        p_t_ref = 0.7
        v_1_ref = 1.0

    if t>5:
        p_t_ref = 0.7
        v_1_ref = 1.1
        
    z_ref = np.copy(np.array([[p_t_ref],
                              [v_1_ref]]))
    
    u_d = control(syst,z_ref)
    
    p_m_ref = u_d[0,0]
    v_f =     u_d[1,0]
           
    events=[{'t_end':t,'v_f':v_f,'p_m_ref':p_m_ref}]
    syst.run(events)
    
    #X_ctrl[it,:] = np.hstack((x_i.T))
    U_ctrl[it,:] = u_d.T
    X_ctrl[it,:] = syst.x_ctrl.T
    X_obs[it,:]  = syst.x_obs.T
    Z_obs[it,:]  = syst.z_obs.T
    it += 1
syst.post();

## Post processing

In [63]:
plt.close('all')
fig, axes = plt.subplots(nrows=2,ncols=2, figsize=(10, 5), frameon=False, dpi=80)

#axes[0,0].plot(syst.T, syst.get_values('omega'), label=f'$\omega$')
#axes[0,0].step(times,X_obs[:,1] + x_d_0[1],   label=f'$\hat \omega$')

axes[0,0].plot(syst.T, syst.get_values('e1q'), label=f"$e'_q$")
axes[0,0].plot(times, X_obs[:,2] + x_d_0[2], label=f"$\hat e'_q$")

axes[0,0].plot(syst.T, syst.get_values('e1d'), label=f"$e'_d$")
axes[0,0].plot(times, X_obs[:,3] + x_d_0[3], label=f"$\hat e'_d$")

#axes[1,1].plot(syst.T, syst.get_values('delta'), label=f'$\delta$')
axes[1,0].plot(syst.T, syst.get_values('p_t'),   label=f'$P_t$')
axes[1,0].plot(syst.T, syst.get_values('v_1'),   label=f'$v_1$')

axes[0,1].step(times, U_ctrl[:,0],   label=f'$p_m^\star$')
axes[0,1].step(times, U_ctrl[:,1],   label=f'$v_f$')

axes[1,1].plot(times,X_ctrl[:,2],   label=f'$\\xi_p$')
axes[1,1].plot(times,X_ctrl[:,3],   label=f'$\\xi_v$')


#axes[1,1].plot(times,X_obs[:,0] + x_d_0[0],   label=f'$X_obs$')
for ax in axes.flatten():
    ax.legend(loc='best')
    ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …